In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the dataset
data = pd.read_json('Dataset.json')

for i in range(0, len(data)):
    if data.annotation[i]['label'][0] == '1':
        data.annotation[i] = 1
    else:
        data.annotation[i] = 0
data['annotation'] = to_categorical(data['annotation'])

Streaming output truncated to the last 5000 lines.
<ipython-input-1-2ab613ea9526>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-1-2ab613ea9526>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-1-2ab613ea9526>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.annotation[i] = 0
<ipython-input-1-2ab613ea9526>:18: SettingWithCopyWarning: 
A value is tryin

In [ ]:
# Preprocessing
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['content'])

# Split data into training and testing sets
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

train_sequences = tokenizer.texts_to_sequences(train_data['content'])
train_padded = pad_sequences(train_sequences, maxlen=200, padding='post')
train_labels = to_categorical(train_data['annotation'])

test_sequences = tokenizer.texts_to_sequences(test_data['content'])
test_padded = pad_sequences(test_sequences, maxlen=200, padding='post')
test_labels = to_categorical(test_data['annotation'])

# Define the model
model = Sequential()
model.add(Embedding(5000, 100, input_length=200))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(64)))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

# Train the model
model.fit(train_padded, train_labels, epochs=30, batch_size=32, validation_data=(test_padded, test_labels), callbacks=[early_stopping, model_checkpoint])
# Evaluate the model
loss, accuracy = model.evaluate(test_padded, test_labels)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

Epoch 1/30
500/500 [==============================] - 112s 214ms/step - loss: 0.6222 - accuracy: 0.6424 - val_loss: 0.2721 - val_accuracy: 0.9433
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


500/500 [==============================] - 105s 209ms/step - loss: 0.4522 - accuracy: 0.7897 - val_loss: 0.1999 - val_accuracy: 0.9300
Epoch 3/30
500/500 [==============================] - 103s 206ms/step - loss: 0.3113 - accuracy: 0.8689 - val_loss: 0.1959 - val_accuracy: 0.9195
Epoch 4/30
500/500 [==============================] - 106s 212ms/step - loss: 0.2147 - accuracy: 0.9160 - val_loss: 0.2548 - val_accuracy: 0.8940
Epoch 5/30
500/500 [==============================] - 102s 205ms/step - loss: 0.1609 - accuracy: 0.9379 - val_loss: 0.2155 - val_accuracy: 0.9180
Epoch 6/30
500/500 [==============================] - 107s 214ms/step - loss: 0.1290 - accuracy: 0.9509 - val_loss: 0.2873 - val_accuracy: 0.9088
Epoch 7/30
500/500 [==============================] - 108s 216ms/step - loss: 0.1084 - accuracy: 0.9612 - val_loss: 0.3298 - val_accuracy: 0.8940
Epoch 8/30
126/126 [==============================] - 6s 49ms/step - loss: 0.3441 - accuracy: 0.8960
Test loss: 0.3440571129322052, Tes

In [ ]:
# Input prediction
while True:
    new_text = input("Enter new text (or 'exit' to quit): ")
    if new_text.lower() == 'exit':
        break

    # Preprocess the new text
    new_sequence = tokenizer.texts_to_sequences([new_text])
    new_padded = pad_sequences(new_sequence, maxlen=200, padding='post')

    # Make prediction
    prediction = model.predict(new_padded)
    predicted_class = np.argmax(prediction, axis=1)[0]

    if predicted_class == 0:
        print("Prediction: Negative")
    else:
        print("Prediction: Positive")

Enter new text (or 'exit' to quit): shes so smart
1/1 [==============================] - 0s 43ms/step
Prediction: Positive
Enter new text (or 'exit' to quit): thats so ugly
1/1 [==============================] - 0s 44ms/step
Prediction: Negative
Enter new text (or 'exit' to quit): exit


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Make predictions on the test data
test_predictions = model.predict(test_padded)

# Convert predictions and true labels from one-hot encoding to their original label indices
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(test_labels, axis=1)

# Generate the classification report
report = classification_report(test_true_labels, test_predicted_labels, digits=2)
print(report)

# Calculate and print the overall accuracy
accuracy = accuracy_score(test_true_labels, test_predicted_labels)
print(f'Overall Accuracy: {accuracy:.10f}')


126/126 [==============================] - 9s 74ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.90      0.95      4001

    accuracy                           0.90      4001
   macro avg       0.50      0.45      0.47      4001
weighted avg       1.00      0.90      0.95      4001

Overall Accuracy: 0.8960259935


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
